# Garbage Classification using Transfer Learning

This notebook demonstrates how to classify garbage images into categories using transfer learning with a pre-trained convolutional neural network.

Dataset: [Garbage Classification dataset on Kaggle](https://www.kaggle.com/datasets/mostafaabla/garbage-classification)


## 1. Setup & Imports

In [ ]:
# Install Kaggle if not already installed
!pip install -q kaggle
# Usual imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix


## 2. Download and Prepare Dataset

**Note:** This notebook does not include the dataset. Please download it from Kaggle using the following commands.

In [ ]:
# Download from Kaggle (uncomment and run if not done already)
# !kaggle datasets download mostafaabla/garbage-classification
# !unzip garbage-classification.zip -d data/

# Set data directory
DATA_DIR = './data/Garbage classification/'

## 3. Data Exploration & Visualization

In [ ]:
import glob
import random
from PIL import Image

classes = os.listdir(DATA_DIR)
print('Classes:', classes)

# Show sample images
plt.figure(figsize=(15,8))
for idx, cls in enumerate(classes):
    img_path = glob.glob(os.path.join(DATA_DIR, cls, '*.jpg'))[0]
    img = Image.open(img_path)
    plt.subplot(2, 4, idx+1)
    plt.imshow(img)
    plt.title(cls)
    plt.axis('off')
plt.tight_layout()
plt.show()

## 4. Data Preparation (ImageDataGenerator)

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.15
)

train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)
val_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=True,
    seed=42
)

## 5. Build Model Using Transfer Learning (MobileNetV2)

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Freeze the base model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

## 6. Train the Model

In [ ]:
EPOCHS = 10
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS
)

## 7. Visualize Training Results

In [ ]:
# Plotting accuracy and loss
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.title('Accuracy')
plt.legend()
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.legend()
plt.show()

## 8. Model Evaluation

In [ ]:
# Evaluate on validation set
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation accuracy: {val_acc:.2f}")

## 9. Confusion Matrix & Classification Report

In [ ]:
# Get predictions and true labels
val_generator.reset()
y_pred = model.predict(val_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = val_generator.classes
labels = list(val_generator.class_indices.keys())

print(classification_report(y_true, y_pred_classes, target_names=labels))
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(8,6))
plt.imshow(cm, cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.xticks(np.arange(len(labels)), labels, rotation=45)
plt.yticks(np.arange(len(labels)), labels)
plt.colorbar()
plt.show()

## 10. Making Predictions on New Images

In [ ]:
def predict_image(img_path):
    img = keras.preprocessing.image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    preds = model.predict(img_array)
    class_idx = np.argmax(preds)
    class_label = labels[class_idx]
    confidence = preds[0][class_idx]
    print(f"Prediction: {class_label} (Confidence: {confidence*100:.2f}%)")
    plt.imshow(img)
    plt.title(f"Prediction: {class_label}")
    plt.axis('off')
    plt.show()

# Example usage:
# predict_image('path/to/your/image.jpg')

---
## Notes
- Make sure to download the dataset using Kaggle CLI as described above.
- Tune hyperparameters and try different pre-trained models for better results.
- If you use this notebook, please cite the original dataset and acknowledge sources as appropriate.